## Introduction
This notebook focuses on implementing and evaluating several classic machine learning models to establish a baseline for predicting antibiotic resistance.  It includes models like Random Forest, Logistic Regression, and LightGBM, which are trained on mass spectrometry data. The notebook covers key steps such as feature selection using SelectFromModel and hyperparameter optimization with grid search. The results of this foundational analysis provide a crucial benchmark for comparing the performance of more advanced models.

### Data Preparation

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import (
    average_precision_score,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score
)

import warnings
warnings.filterwarnings('ignore')

class BCOLORS:
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

In [2]:
%load_ext cuml.accel

Load Dataset

In [3]:
if os.path.exists('../data/s_aureus_driams_a_bin3_2000_20000Da.csv'):
    df = pd.read_csv('../data/s_aureus_driams_a_bin3_2000_20000Da.csv')
else:
    df = pd.read_csv('https://media.githubusercontent.com/media/xlopez-ml/DL-AMR/master/datasets/driams_a/s_aureus_driams_a_bin3_2000_20000Da.csv')

In [4]:
df.head()

,2000,2003,2006,2009,2012,2015,2018,2021,2024,2027,...,19991,19994,19997,code,species,Oxacillin,Clindamycin,Ceftriaxone,Ciprofloxacin,Fusidic acid
0,951.428571,826.125000,944.857143,898.428571,1007.714286,936.000000,828.571429,812.000000,782.714286,741.625000,...,35.5,20.5,27.342857,029f0abf-1664-424f-88cb-11c9c8af2b11,Staphylococcus aureus,0.0,0.0,0.0,0.0,1.0
1,784.714286,701.125000,681.571429,762.142857,737.571429,733.000000,735.857143,659.571429,802.714286,554.375000,...,30.0,40.5,34.695238,08149af1-10f5-4f02-81d8-3d46d66d4a7a,Staphylococcus aureus,0.0,0.0,0.0,0.0,0.0
2,50.875000,89.142857,56.142857,151.285714,132.714286,110.428571,47.285714,63.428571,100.375000,52.142857,...,2.5,30.5,11.288462,08fe3876-ecee-4ddc-9aa9-a84f605757f7,Staphylococcus aureus,0.0,0.0,0.0,0.0,0.0
3,843.285714,816.500000,734.428571,1027.142857,979.428571,875.571429,912.142857,714.428571,945.428571,886.500000,...,15.5,8.0,20.057143,0a9fdd4b-7180-47c0-849e-cb32d46da94c,Staphylococcus aureus,0.0,0.0,0.0,0.0,0.0
4,307.857143,295.000000,246.714286,181.571429,258.000000,318.714286,295.714286,300.857143,278.142857,256.125000,...,25.5,32.0,24.990476,0d82f4c4-473c-49d8-8c85-e3e36e96d047,Staphylococcus aureus,1.0,0.0,1.0,1.0,0.0


In [5]:
antibiotics = df.columns[-5:].to_list()
antibiotics

['Oxacillin', 'Clindamycin', 'Ceftriaxone', 'Ciprofloxacin', 'Fusidic acid']

Create Dataset

In [6]:
def create_dataset(df, antibiotic, max_length):
    antibiotic_data = df.iloc[:, :max_length].join(df[antibiotic])
    antibiotic_data = antibiotic_data.dropna(how='any').reset_index(drop=True)

    model_data = antibiotic_data.drop(antibiotic, axis=1)
    model_labels = antibiotic_data[[antibiotic]]

    clf = RandomForestClassifier(n_jobs=12, random_state=0)
    selector = SelectFromModel(estimator=clf, threshold="1.25*mean")
    selector.fit(model_data, np.ravel(model_labels))

    model_data = model_data.loc[:, selector.get_support()]

    model_overall = model_data.assign(label=model_labels)
    

    return model_overall

### Model Selection

In [7]:
def evaluation(y_test, y_pred_proba, y_pred, antibiotic, clf_name):
    auprc = average_precision_score(y_test, y_pred_proba)
    auroc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    return {
        'Antibiotic': antibiotic,
        'clf': clf_name,
        'AUPRC': auprc,
        'AUROC': auroc,
        'F1 Score': f1,
        'Precision': precision,
        'Recall': recall
        }

In [8]:
classifiers = {
    "Random Forest": RandomForestClassifier(random_state=0, n_jobs=-1),
    "Logistic Regression": LogisticRegression(random_state=0, n_jobs=-1),
    "Support Vector Machine": SVC(probability=True, random_state=0),
    "Naive Bayes": GaussianNB(),
    "LightGBM": LGBMClassifier(random_state=0, n_jobs=12, verbose=-1),
    "Multilayer Perceptron": MLPClassifier(
        hidden_layer_sizes=(256, 128),
        random_state=0,
        max_iter=500,
    ),
}

scores = []
for antibiotic in antibiotics:
    antibiotic_data = create_dataset(df, antibiotic, 6000)
    model_data = antibiotic_data.drop("label", axis=1)
    model_labels = antibiotic_data[["label"]]

    if antibiotic == "Logistic Regression" or antibiotic == "Support Vector Machine":
        scaler = StandardScaler()
        scaler.fit(model_data)
        model_data = scaler.transform(model_data)

    X_train, X_test, y_train, y_test = train_test_split(
        model_data, model_labels, test_size=0.2, random_state=0, stratify=model_labels
    )

    print(f"Currently on {antibiotic}.")

    for name, clf in classifiers.items():
        clf.fit(X_train, np.ravel(y_train))

        if hasattr(clf, "predict_proba"):
            y_scores = clf.predict_proba(X_test)[:, 1]
        else:
            y_scores = clf.decision_function(X_test)

        y_pred = clf.predict(X_test)

        auroc = roc_auc_score(np.ravel(y_test), y_scores)
        scores.append(evaluation(y_test, y_scores, y_pred, antibiotic, name))

        print(f"Antibiotic: {antibiotic}, Model: {name}, AUROC: {auroc:.4f}")
        with open("model_selection.csv", "a") as log:
            log.write(f'"{antibiotic}","{name}",{auroc:.4f}\n')
    print('\n', '-'*100, '\n')

Currently on Oxacillin.
Antibiotic: Oxacillin, Model: Random Forest, AUROC: 0.8113
Antibiotic: Oxacillin, Model: Logistic Regression, AUROC: 0.8620
Antibiotic: Oxacillin, Model: Support Vector Machine, AUROC: 0.8366
Antibiotic: Oxacillin, Model: Naive Bayes, AUROC: 0.5682
Antibiotic: Oxacillin, Model: LightGBM, AUROC: 0.8723
Antibiotic: Oxacillin, Model: Multilayer Perceptron, AUROC: 0.8118

 ---------------------------------------------------------------------------------------------------- 

Currently on Clindamycin.
Antibiotic: Clindamycin, Model: Random Forest, AUROC: 0.5840
Antibiotic: Clindamycin, Model: Logistic Regression, AUROC: 0.7184
Antibiotic: Clindamycin, Model: Support Vector Machine, AUROC: 0.7150
Antibiotic: Clindamycin, Model: Naive Bayes, AUROC: 0.5440
Antibiotic: Clindamycin, Model: LightGBM, AUROC: 0.6677
Antibiotic: Clindamycin, Model: Multilayer Perceptron, AUROC: 0.6568

 -------------------------------------------------------------------------------------------

In [9]:
scores_df = pd.DataFrame(scores)
scores_df

,Antibiotic,clf,AUPRC,AUROC,F1 Score,Precision,Recall
0,Oxacillin,Random Forest,0.623975,0.811279,0.357542,0.941176,0.220690
1,Oxacillin,Logistic Regression,0.725111,0.862046,0.679389,0.760684,0.613793
2,Oxacillin,Support Vector Machine,0.668424,0.836632,0.104575,1.000000,0.055172
3,Oxacillin,Naive Bayes,0.218019,0.568245,0.356050,0.222997,0.882759
4,Oxacillin,LightGBM,0.748658,0.872284,0.627273,0.920000,0.475862
5,Oxacillin,Multilayer Perceptron,0.684341,0.811779,0.519608,0.898305,0.365517
6,Clindamycin,Random Forest,0.227192,0.584044,0.055556,0.750000,0.028846
7,Clindamycin,Logistic Regression,0.434788,0.718431,0.417178,0.576271,0.326923
8,Clindamycin,Support Vector Machine,0.323544,0.715032,0.000000,0.000000,0.000000
9,Clindamycin,Naive Bayes,0.174204,0.544032,0.250923,0.203593,0.326923


### Model Fine-Tuning

In [ ]:
classifier_tests = {
    "Random Forest": {
        "clf": RandomForestClassifier(),
        "param_grid": {
            "random_state": [0],
            "n_estimators": [100, 200, 400],
            "max_features": ["sqrt", "log2", 60, 100],
        },
        "needs_scaled_data": False,
    },
    "Logistic Regression": {
        "clf": LogisticRegression(),
        "param_grid": {
            "random_state": [0],
            "C": [0.001, 0.01, 0.1],
            "penalty": ["l1", "l2"],
            "solver": ["lbfgs", "saga", "liblinear", "newton-cg"],
            "max_iter": [1000]
        },
        "needs_scaled_data": True,
    },
    "Support Vector Machine": {
        "clf": SVC(),
        "param_grid": {
            "random_state": [0],
            "probability": [True],
            "C": [0.1, 1, 10, 100],
            "kernel": ["rbf"],
            "gamma": ["scale", "auto"],
        },
        "needs_scaled_data": True,
    },
    "Naive Bayes": {
        "clf": GaussianNB(),
        "param_grid": {},
        "needs_scaled_data": False,
    },
    "LightGBM": {
        "clf": LGBMClassifier(),
        "param_grid": {
            "random_state": [0],
            "verbose": [-1],
            "objective": ["binary"],
            "boosting_type": ["gbdt"],
            "n_estimators": [100, 200, 400],
            "learning_rate": [0.01, 0.1, 1],
        },
        "needs_scaled_data": False,
    },
    "Multilayer Perceptron": {
        "clf": MLPClassifier(),
        "param_grid": {
            "random_state": [0],
            "hidden_layer_sizes": [
                (512, 256, 128),
                (512, 128, 64),
                (256, 64),
                (256, 128),
            ],
            "activation": ["relu"],
            "alpha": [0.0001],
            "max_iter": [1000],
        },
        "needs_scaled_data": False,
    },
}

for antibiotic in antibiotics:
    antibiotic_data = create_dataset(df, antibiotic, 6000)
    model_data = antibiotic_data.drop("label", axis=1)
    model_labels = np.ravel(antibiotic_data[["label"]])


    for name, test in classifier_tests.items():
        current_model_data = model_data.copy()
        if test["needs_scaled_data"]:
            scaler = StandardScaler()
            scaler.fit(current_model_data)
            current_model_data = scaler.transform(current_model_data)

        try:
            clf = test["clf"]
            grid = GridSearchCV(
                estimator=clf,
                param_grid=test["param_grid"],
                cv=5,
                scoring="roc_auc",
                verbose=1,
                n_jobs=-1,
            )
            grid.fit(current_model_data, model_labels)

            if grid.best_score_ > 0.7:
                print(
                    BCOLORS.OKGREEN
                    + f"Antibiotic: {antibiotic}, Model: {name}, AUROC: {grid.best_score_}, Params: {grid.best_params_}\n"
                    + BCOLORS.ENDC
                )
            else:
                print(
                    f"Antibiotic: {antibiotic}, Model: {name}, AUROC: {grid.best_score_}, Params: {grid.best_params_}\n"
                )
            with open("model_tuning.txt", "a") as text_file:
                text_file.write(
                    f"Antibiotic: {antibiotic}, Model: {name}, AUROC: {grid.best_score_}, Params: {grid.best_params_}\n"
                )
        except Exception as e:
            print(
                BCOLORS.FAIL + f"[ERROR] {e}, Antibiotic: {antibiotic}" + BCOLORS.ENDC
            )

### Model Evaluation

In [11]:
BEST_CLASSIFIERS = {
    "Oxacillin": {
        "clf_name": "Logistic Regression",
        "clf": LogisticRegression(),
        "params": {'C': 0.1,
                   'max_iter': 1000,
                   'penalty': 'l2',
                   'random_state': 0,
                   'solver': 'liblinear'},
        "needs_scaled_data": True,
    },
    "Clindamycin": {
        "clf_name": "Logistic Regression",
        "clf": LogisticRegression(),
        "params": {'C': 0.1,
                   'max_iter': 1000,
                   'penalty': 'l2',
                   'random_state': 0,
                   'solver': 'liblinear'},
        "needs_scaled_data": True,
    },
    "Ceftriaxone": {
        "clf_name": "Support Vector Machine",
        "clf": SVC(),
        "params": {'C': 100,
                   'gamma': 'auto',
                   'kernel': 'rbf',
                   'probability': True,
                   'random_state': 0},
        "needs_scaled_data": True,
    },
    "Ciprofloxacin": {
        "clf_name": "Logistic Regression",
        "clf": LogisticRegression(),
        "params": {'C': 0.1,
                   'max_iter': 1000,
                   'penalty': 'l2',
                   'random_state': 0,
                   'solver': 'liblinear'},
        "needs_scaled_data": True,
    },
    "Fusidic acid": {
        "clf_name": "Logistic Regression",
        "clf": LogisticRegression(),
        "params": {'C': 0.1,
                   'max_iter': 1000,
                   'penalty': 'l2',
                   'random_state': 0,
                   'solver': 'lbfgs'},
        "needs_scaled_data": True,
    },
    }

In [12]:
scores = []

for antibiotic, test in BEST_CLASSIFIERS.items():
    antibiotic_data = create_dataset(df, antibiotic, 6000)
    model_data = antibiotic_data.drop("label", axis=1)
    model_labels = np.ravel(antibiotic_data[["label"]])

    current_model_data = model_data.copy()
    if test["needs_scaled_data"]:
        scaler = StandardScaler()
        scaler.fit(current_model_data)
        current_model_data = scaler.transform(current_model_data)

    clf = test["clf"]
    clf.set_params(**test["params"])
    if test["clf_name"] != "Support Vector Machine":
        clf.set_params(n_jobs=-1)

    X_train, X_test, y_train, y_test = train_test_split(
        current_model_data, model_labels, test_size=0.2, stratify=model_labels, random_state=0
    )
    clf.fit(X_train, y_train)

    y_pred_proba = clf.predict_proba(X_test)[:, 1] 
    y_pred = clf.predict(X_test)
    scores.append(evaluation(y_test, y_pred_proba, y_pred, antibiotic, test['clf_name']))
    

In [13]:
scores_df = pd.DataFrame(scores)
scores_df

,Antibiotic,clf,AUPRC,AUROC,F1 Score,Precision,Recall
0,Oxacillin,Logistic Regression,0.781500,0.892760,0.612613,0.883117,0.468966
1,Clindamycin,Logistic Regression,0.405795,0.736214,0.297872,0.567568,0.201923
2,Ceftriaxone,Support Vector Machine,0.797463,0.908979,0.708171,0.791304,0.640845
3,Ciprofloxacin,Logistic Regression,0.606387,0.839622,0.497238,0.775862,0.365854
4,Fusidic acid,Logistic Regression,0.336680,0.816556,0.193548,0.545455,0.117647
